In [ ]:
#import requests
#import bs4
#from bs4 import BeautifulSoup
from tqdm import tqdm

import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings

pd.set_option('display.max_columns', 500)   # to display 500 columns
pd.set_option('display.max_rows', 500) # to display 500 rows

import time
from datetime import datetime

from sklearn.preprocessing import StandardScaler,

In [2]:
import psycopg2

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to connect to the northwind database
database =   'cms_claims' # 'cms_medicare_claims'  #

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

In [3]:
# Check Tables in the Database
query = """
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname='public'
"""

pd.read_sql(query, con=conn)

,tablename
0,icd9_diagonsis
1,icd9_procedures
2,hcpcs
3,beneficiary2008
4,beneficiary2009
5,beneficiary2010
6,inpatient_claims
7,outpatient_claims
8,prescription_drug_events
9,final2008_2009


In [4]:
datetime.now()

datetime.datetime(2020, 5, 12, 15, 26, 25, 269332)

In [5]:
# Replacing  Nulls with 0

query = """

SELECT 

     "DESYNPUF_ID"
    , "BENE_BIRTH_DT"
    , COALESCE("BENE_DEATH_DT", '2100-01-01') as "BENE_DEATH_DT"
    , COALESCE("BENE_SEX_IDENT_CD", 0) as "BENE_SEX_IDENT_CD"
    , COALESCE("BENE_RACE_CD", 0) as "BENE_RACE_CD"
    , cast((CASE when "END_STAGE_RENAL2008" like 'Y' then '1' else '0' END) as SMALLINT) as "END_STAGE_RENAL2008"
    , cast((CASE when "END_STAGE_RENAL2009" like 'Y' then '1' else '0' END) as smallint) as "END_STAGE_RENAL2009"
    , COALESCE("SP_ALZHDMTA2008", 0) as "SP_ALZHDMTA2008"
    , COALESCE("SP_ALZHDMTA2009", 0) as "SP_ALZHDMTA2009"
    , COALESCE("SP_CHF2008", 0) as "SP_CHF2008"
    , COALESCE("SP_CHF2009", 0) as "SP_CHF2009"
    , COALESCE("SP_CHRNKIDN2008", 0) as "SP_CHRNKIDN2008"
    , COALESCE("SP_CHRNKIDN2009", 0) as "SP_CHRNKIDN2009"
    , COALESCE("SP_CNCR2008", 0) as "SP_CNCR2008"
    , COALESCE("SP_CNCR2009", 0) as "SP_CNCR2009"
    , COALESCE("SP_COPD2008", 0) as "SP_COPD2008"
    , COALESCE("SP_COPD2009", 0) as "SP_COPD2009"
    , COALESCE("SP_DEPRESSN2008", 0) as "SP_DEPRESSN2008"
    , COALESCE("SP_DEPRESSN2009", 0) as "SP_DEPRESSN2009"
    , COALESCE("SP_DIABETES2008", 0) as "SP_DIABETES2008"
    , COALESCE("SP_DIABETES2009", 0) as "SP_DIABETES2009"
    , COALESCE("SP_ISCHMCHT2008", 0) as "SP_ISCHMCHT2008"
    , COALESCE("SP_ISCHMCHT2009", 0) as "SP_ISCHMCHT2009"
    , COALESCE("SP_OSTEOPRS2008", 0) as "SP_OSTEOPRS2008"
    , COALESCE("SP_OSTEOPRS2009", 0) as "SP_OSTEOPRS2009"
    , COALESCE("SP_RA_OA2008", 0) as "SP_RA_OA2008"
    , COALESCE("SP_RA_OA2009", 0) as "SP_RA_OA2009"
    , COALESCE("SP_STRKETIA2008", 0) as "SP_STRKETIA2008"
    , COALESCE("TOTAL_DIAGNOSIS2008", 0) as "TOTAL_DIAGNOSIS2008"
    , COALESCE("TOTAL_DIAGNOSIS2009", 0) as "TOTAL_DIAGNOSIS2009"
    , COALESCE("TOTAL_PAYMENT2008", 0) as "TOTAL_PAYMENT2008"
    , COALESCE("TOTAL_PAYMENT2009", 0) as "TOTAL_PAYMENT2009"
    , COALESCE("CHANGE_IN_PAYMEMT", 0) as "CHANGE_IN_PAYMEMT"
    , COALESCE("CHANGE_IN_DIANOSIS", 0) as "CHANGE_IN_DIAGNOSIS"
    , COALESCE("DEAD", 0) as "DEAD"
    , COALESCE("AGE", 0) as "AGE"
    , COALESCE("LOS", 0) as "LOS"
    , COALESCE("NUM_INPT_ADM", 0) as "NUM_INPT_ADM"
    , COALESCE("TOTAL_INPT_COST", 0) as "TOTAL_INPT_COST"
    , COALESCE("TOTAL_INPT_DIAGNOSIS", 0) as "TOTAL_INPT_DIAGNOSIS"
    , COALESCE("TOTAL_INPT_PROCS", 0) as "TOTAL_INPT_PROCS"
    , COALESCE("READMIT7", 0) as "READMIT7" 
    , COALESCE("READMIT30", 0) as "READMIT30"
    , COALESCE("READMIT60", 0) as "READMIT60"
    , COALESCE("READMIT90", 0) as "READMIT90"
    , COALESCE("NUM_OPD_VISIT", 0) as "NUM_OPD_VISIT"
    , COALESCE("TOTAL_OPD_COST", 0) as "TOTAL_OPD_COST"
    , COALESCE("TOTAL_OPD_DIAGNOSIS", 0) as "TOTAL_OPD_DIAGNOSIS"
    , COALESCE("TOTAL_OPD_PROCS", 0) as "TOTAL_OPD_PROCS"
    , COALESCE("TOTAL_OPD_HCPCS", 0) as "TOTAL_OPD_HCPCS"
    , COALESCE("TOTAL_QTY_DSPNSD_NUM", 0) as "TOTAL_QTY_DSPNSD_NUM"
    , COALESCE("PTNT_PAY_RX_AMT", 0) as "PTNT_PAY_RX_AMT"
    , COALESCE("TOT_RX_CST_AMT", 0) as "TOT_RX_CST_AMT"
    , COALESCE("INPT_DIAGS", '') as "INPT_DIAGS"
    , COALESCE("INPT_PROCS", '') as "INPT_PROCS"
    , COALESCE("OPD_DIAGS", '') as "OPD_DIAGS"
    , COALESCE("OPD_PROCS", '') as "OPD_PROCS"
    , COALESCE("OPD_HCPCS", '') as "OPD_HCPCS"
    , CASE when "READMIT7" = 0.0 then 0 else 1 END as "READMIT7_FLAG"  
    , CASE when "READMIT30" = 0.0 then 0 else 1 END as "READMIT30_FLAG" 
    , CASE when "READMIT60" = 0.0 then 0 else 1 END as "READMIT60_FLAG" 
    , CASE when "READMIT90" = 0.0 then 0 else 1 END as "READMIT90_FLAG" 

FROM final2008_2009v1  
"""

df = pd.read_sql(query, conn) 
# df is patients who are in 2008 and 2009 benefciiary, unfortunately 2010 doesnt have common benefciary patients

print(df.shape)
df.head(2)

(458328, 62)


,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,END_STAGE_RENAL2008,END_STAGE_RENAL2009,SP_ALZHDMTA2008,SP_ALZHDMTA2009,SP_CHF2008,SP_CHF2009,SP_CHRNKIDN2008,SP_CHRNKIDN2009,SP_CNCR2008,SP_CNCR2009,SP_COPD2008,SP_COPD2009,SP_DEPRESSN2008,SP_DEPRESSN2009,SP_DIABETES2008,SP_DIABETES2009,SP_ISCHMCHT2008,SP_ISCHMCHT2009,SP_OSTEOPRS2008,SP_OSTEOPRS2009,SP_RA_OA2008,SP_RA_OA2009,SP_STRKETIA2008,TOTAL_DIAGNOSIS2008,TOTAL_DIAGNOSIS2009,TOTAL_PAYMENT2008,TOTAL_PAYMENT2009,CHANGE_IN_PAYMEMT,CHANGE_IN_DIAGNOSIS,DEAD,AGE,LOS,NUM_INPT_ADM,TOTAL_INPT_COST,TOTAL_INPT_DIAGNOSIS,TOTAL_INPT_PROCS,READMIT7,READMIT30,READMIT60,READMIT90,NUM_OPD_VISIT,TOTAL_OPD_COST,TOTAL_OPD_DIAGNOSIS,TOTAL_OPD_PROCS,TOTAL_OPD_HCPCS,TOTAL_QTY_DSPNSD_NUM,PTNT_PAY_RX_AMT,TOT_RX_CST_AMT,INPT_DIAGS,INPT_PROCS,OPD_DIAGS,OPD_PROCS,OPD_HCPCS,READMIT7_FLAG,READMIT30_FLAG,READMIT60_FLAG,READMIT90_FLAG
0,00000B48BCF4AD29,1923-09-01,2100-01-01,2,5,0,0,1,1,1,1,1,2,1,2,2,2,1,1,1,1,1,1,1,1,2,2,1,9,6,93452.0,8750.0,-84702.0,-3,0,86.0,2419200000000000,3.0,84072.0,27.0,11.0,2.0,2.0,2.0,2.0,10.0,3210.0,27.0,0.0,47.0,210.0,10.0,200.0,4241 2760 7802 2763 42821 2762 41410 3970 511...,3521 9971 5601 V451 V451 41071 3615 53081 428...,72999 72981 2721 V1250 2330 79...,...,93971 G0103 80076 80053 80053 856...,1,1,1,1
1,0000141F2FECE9BC,1974-04-01,2100-01-01,2,1,0,0,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,0,1,0.0,5308.0,5308.0,1,0,35.0,172800000000000,1.0,5108.0,9.0,1.0,0.0,0.0,0.0,0.0,30.0,7870.0,99.0,0.0,67.0,9930.0,1740.0,13490.0,5789 79029 5363 5990 3569 2449 2851 311 2869,4516,32731 78079 5990 53081 OTHER 2...,...,92557 A0436 80053 A95...,0,0,0,0


In [6]:
#df.READMIT90_FLAG.unique()
datetime.now()

datetime.datetime(2020, 5, 12, 15, 26, 42, 482821)

In [7]:
# Checking for nulls

df.isnull().sum()

DESYNPUF_ID             0
BENE_BIRTH_DT           0
BENE_DEATH_DT           0
BENE_SEX_IDENT_CD       0
BENE_RACE_CD            0
END_STAGE_RENAL2008     0
END_STAGE_RENAL2009     0
SP_ALZHDMTA2008         0
SP_ALZHDMTA2009         0
SP_CHF2008              0
SP_CHF2009              0
SP_CHRNKIDN2008         0
SP_CHRNKIDN2009         0
SP_CNCR2008             0
SP_CNCR2009             0
SP_COPD2008             0
SP_COPD2009             0
SP_DEPRESSN2008         0
SP_DEPRESSN2009         0
SP_DIABETES2008         0
SP_DIABETES2009         0
SP_ISCHMCHT2008         0
SP_ISCHMCHT2009         0
SP_OSTEOPRS2008         0
SP_OSTEOPRS2009         0
SP_RA_OA2008            0
SP_RA_OA2009            0
SP_STRKETIA2008         0
TOTAL_DIAGNOSIS2008     0
TOTAL_DIAGNOSIS2009     0
TOTAL_PAYMENT2008       0
TOTAL_PAYMENT2009       0
CHANGE_IN_PAYMEMT       0
CHANGE_IN_DIAGNOSIS     0
DEAD                    0
AGE                     0
LOS                     0
NUM_INPT_ADM            0
TOTAL_INPT_C

In [8]:
#Checking data types

df.dtypes

DESYNPUF_ID                     object
BENE_BIRTH_DT           datetime64[ns]
BENE_DEATH_DT           datetime64[ns]
BENE_SEX_IDENT_CD                int64
BENE_RACE_CD                     int64
END_STAGE_RENAL2008              int64
END_STAGE_RENAL2009              int64
SP_ALZHDMTA2008                  int64
SP_ALZHDMTA2009                  int64
SP_CHF2008                       int64
SP_CHF2009                       int64
SP_CHRNKIDN2008                  int64
SP_CHRNKIDN2009                  int64
SP_CNCR2008                      int64
SP_CNCR2009                      int64
SP_COPD2008                      int64
SP_COPD2009                      int64
SP_DEPRESSN2008                  int64
SP_DEPRESSN2009                  int64
SP_DIABETES2008                  int64
SP_DIABETES2009                  int64
SP_ISCHMCHT2008                  int64
SP_ISCHMCHT2009                  int64
SP_OSTEOPRS2008                  int64
SP_OSTEOPRS2009                  int64
SP_RA_OA2008             

In [9]:
#df.columns

In [10]:
# Some columns still have None, replacing with space

df.INPT_DIAGS.fillna('', inplace=True)
#df.INPT_DIAGS.values

df.INPT_PROCS.fillna('', inplace=True)
df.OPD_DIAGS.fillna('', inplace=True)
df.OPD_PROCS.fillna('', inplace=True)

In [11]:
# Combining all the diagnosis and procedure codes from inpatient and outpatient visits (as some codes may be 
#repated and we want total counts, not duplicate features)

df_ICD_codes = df.INPT_DIAGS + df.INPT_PROCS + df.OPD_DIAGS + df.OPD_PROCS
df_ICD_codes.shape

(458328,)

In [32]:
datetime.now()

datetime.datetime(2020, 5, 12, 15, 55, 36, 269211)

In [33]:
# it takes around 1 min for 0.5 million rows, creating 15k columns

from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(token_pattern='\w+')

cvec.fit(df_ICD_codes.values)
ICD_codes_vectorized = cvec.transform(df_ICD_codes.values)

#df_ICD_codes_vectorized = pd.DataFrame(cvec.transform(df_ICD_codes.values).toarray(),columns=cvec.get_feature_names())#.astype('int8')

#print(df_ICD_codes_vectorized.shape)
#df_ICD_codes_vectorized.head(2)

In [34]:
ICD_codes_vectorized.shape

(458328, 14107)

In [35]:
df_ICD_codes_vectorized

<458328x14107 sparse matrix of type '<class 'numpy.int64'>'
	with 9683302 stored elements in Compressed Sparse Row format>

In [37]:
#df_ICD_codes_vectorized = df_ICD_codes_vectorized.astype('int8')
#df_ICD_codes_vectorized['0023'].dtypes

In [38]:
datetime.now()

datetime.datetime(2020, 5, 12, 15, 55, 57, 349632)

In [ ]:
scaler = StandardScaler()
ICD_codes_vectorized = scaler.fit_transform(ICD_codes_vectorized)

In [42]:
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD

Truncated_pca = TruncatedSVD(n_components=500) # 600 return approx 82% of the variance

# PCA fit-tranform 
ICD_codes_vectorized_pca = Truncated_pca.fit_transform(ICD_codes_vectorized) #.toarray()

# Converting into DataFrame
PCA_col_dict = {'PCA'+str(i+1): Truncated_pca.components_[i]
            for i in range(len(Truncated_pca.components_))}

df_ICD_codes_vectorized_pca = pd.DataFrame(ICD_codes_vectorized_pca, columns=PCA_col_dict) 
print(df_ICD_codes_vectorized_pca.shape)

df_ICD_codes_vectorized_pca.head(2)

(458328, 600)


,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30,PCA31,PCA32,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,PCA41,PCA42,PCA43,PCA44,PCA45,PCA46,PCA47,PCA48,PCA49,PCA50,PCA51,PCA52,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,PCA61,PCA62,PCA63,PCA64,PCA65,PCA66,PCA67,PCA68,PCA69,PCA70,PCA71,PCA72,PCA73,PCA74,PCA75,PCA76,PCA77,PCA78,PCA79,PCA80,PCA81,PCA82,PCA83,PCA84,PCA85,PCA86,PCA87,PCA88,PCA89,PCA90,PCA91,PCA92,PCA93,PCA94,PCA95,PCA96,PCA97,PCA98,PCA99,PCA100,PCA101,PCA102,PCA103,PCA104,PCA105,PCA106,PCA107,PCA108,PCA109,PCA110,PCA111,PCA112,PCA113,PCA114,PCA115,PCA116,PCA117,PCA118,PCA119,PCA120,PCA121,PCA122,PCA123,PCA124,PCA125,PCA126,PCA127,PCA128,PCA129,PCA130,PCA131,PCA132,PCA133,PCA134,PCA135,PCA136,PCA137,PCA138,PCA139,PCA140,PCA141,PCA142,PCA143,PCA144,PCA145,PCA146,PCA147,PCA148,PCA149,PCA150,PCA151,PCA152,PCA153,PCA154,PCA155,PCA156,PCA157,PCA158,PCA159,PCA160,PCA161,PCA162,PCA163,PCA164,PCA165,PCA166,PCA167,PCA168,PCA169,PCA170,PCA171,PCA172,PCA173,PCA174,PCA175,PCA176,PCA177,PCA178,PCA179,PCA180,PCA181,PCA182,PCA183,PCA184,PCA185,PCA186,PCA187,PCA188,PCA189,PCA190,PCA191,PCA192,PCA193,PCA194,PCA195,PCA196,PCA197,PCA198,PCA199,PCA200,PCA201,PCA202,PCA203,PCA204,PCA205,PCA206,PCA207,PCA208,PCA209,PCA210,PCA211,PCA212,PCA213,PCA214,PCA215,PCA216,PCA217,PCA218,PCA219,PCA220,PCA221,PCA222,PCA223,PCA224,PCA225,PCA226,PCA227,PCA228,PCA229,PCA230,PCA231,PCA232,PCA233,PCA234,PCA235,PCA236,PCA237,PCA238,PCA239,PCA240,PCA241,PCA242,PCA243,PCA244,PCA245,PCA246,PCA247,PCA248,PCA249,PCA250,...,PCA351,PCA352,PCA353,PCA354,PCA355,PCA356,PCA357,PCA358,PCA359,PCA360,PCA361,PCA362,PCA363,PCA364,PCA365,PCA366,PCA367,PCA368,PCA369,PCA370,PCA371,PCA372,PCA373,PCA374,PCA375,PCA376,PCA377,PCA378,PCA379,PCA380,PCA381,PCA382,PCA383,PCA384,PCA385,PCA386,PCA387,PCA388,PCA389,PCA390,PCA391,PCA392,PCA393,PCA394,PCA395,PCA396,PCA397,PCA398,PCA399,PCA400,PCA401,PCA402,PCA403,PCA404,PCA405,PCA406,PCA407,PCA408,PCA409,PCA410,PCA411,PCA412,PCA413,PCA414,PCA415,PCA416,PCA417,PCA418,PCA419,PCA420,PCA421,PCA422,PCA423,PCA424,PCA425,PCA426,PCA427,PCA428,PCA429,PCA430,PCA431,PCA432,PCA433,PCA434,PCA435,PCA436,PCA437,PCA438,PCA439,PCA440,PCA441,PCA442,PCA443,PCA444,PCA445,PCA446,PCA447,PCA448,PCA449,PCA450,PCA451,PCA452,PCA453,PCA454,PCA455,PCA456,PCA457,PCA458,PCA459,PCA460,PCA461,PCA462,PCA463,PCA464,PCA465,PCA466,PCA467,PCA468,PCA469,PCA470,PCA471,PCA472,PCA473,PCA474,PCA475,PCA476,PCA477,PCA478,PCA479,PCA480,PCA481,PCA482,PCA483,PCA484,PCA485,PCA486,PCA487,PCA488,PCA489,PCA490,PCA491,PCA492,PCA493,PCA494,PCA495,PCA496,PCA497,PCA498,PCA499,PCA500,PCA501,PCA502,PCA503,PCA504,PCA505,PCA506,PCA507,PCA508,PCA509,PCA510,PCA511,PCA512,PCA513,PCA514,PCA515,PCA516,PCA517,PCA518,PCA519,PCA520,PCA521,PCA522,PCA523,PCA524,PCA525,PCA526,PCA527,PCA528,PCA529,PCA530,PCA531,PCA532,PCA533,PCA534,PCA535,PCA536,PCA537,PCA538,PCA539,PCA540,PCA541,PCA542,PCA543,PCA544,PCA545,PCA546,PCA547,PCA548,PCA549,PCA550,PCA551,PCA552,PCA553,PCA554,PCA555,PCA556,PCA557,PCA558,PCA559,PCA560,PCA561,PCA562,PCA563,PCA564,PCA565,PCA566,PCA567,PCA568,PCA569,PCA570,PCA571,PCA572,PCA573,PCA574,PCA575,PCA576,PCA577,PCA578,PCA579,PCA580,PCA581,PCA582,PCA583,PCA584,PCA585,PCA586,PCA587,PCA588,PCA589,PCA590,PCA591,PCA592,PCA593,PCA594,PCA595,PCA596,PCA597,PCA598,PCA599,PCA600
0,3.647593,-0.782340,-0.935462,-0.046088,1.017342,1.831235,0.164655,-0.692281,0.362287,-0.577394,-0.369858,1.329667,-1.084824,0.006693,1.390489,-0.882084,-0.491234,-0.335683,0.696851,0.094493,-0.462910,0.064547,0.175329,-0.170556,1.008885,-1.333923,0.372988,0.194541,-0.762862,-0.084298,-0.061375,0.634967,-0.057567,-0.118257,0.080684,0.471468,0.235070,-0.206037,0.257731,-0.142542,-0.105404,-0.237514,-0.312377,-0.281149,-0.387043,-0.242147,-0.000794,-0.458357,0.435621,0.121936,-0.405941,-0.110268,-0.018663,-0.075231,-0.089828,-0.047926,0.274208,0.933219,-0.700989,0.909341,-0.372295,-0.288081,-0.288

In [43]:
explained_variance = pca.explained_variance_ratio_
explained_variance.cumsum()

array([0.13249586, 0.23086236, 0.25129004, 0.26658109, 0.28090683,
       0.29417533, 0.30488829, 0.31477107, 0.3229523 , 0.33084135,
       0.33844126, 0.34535122, 0.35177414, 0.35793964, 0.36387108,
       0.3694234 , 0.37473184, 0.37962455, 0.38395244, 0.38780088,
       0.39146145, 0.39489332, 0.39812454, 0.40128031, 0.4043001 ,
       0.40713407, 0.40990275, 0.41250834, 0.41506488, 0.41758903,
       0.42008884, 0.42251798, 0.42490656, 0.42727834, 0.42961084,
       0.4319034 , 0.43412642, 0.43633244, 0.43852051, 0.44068598,
       0.44282665, 0.44494754, 0.44704831, 0.44912642, 0.45116976,
       0.45320086, 0.4552234 , 0.45722471, 0.45922328, 0.46120555,
       0.46314724, 0.46503934, 0.46688847, 0.46873193, 0.47056048,
       0.47234379, 0.47411153, 0.47586649, 0.47760902, 0.47934492,
       0.48103269, 0.48269755, 0.48435677, 0.48599868, 0.48762779,
       0.48921988, 0.49080323, 0.49236611, 0.49391804, 0.4954546 ,
       0.49697095, 0.49847077, 0.49995375, 0.50142738, 0.50289

In [44]:
datetime.now()

datetime.datetime(2020, 5, 12, 16, 6, 29, 9599)

In [45]:
# Adding patient ID

#df_ICD_codes_vectorized['DESYNPUF_ID'] = df['DESYNPUF_ID']
df_ICD_codes_vectorized_pca['DESYNPUF_ID'] = df['DESYNPUF_ID']

print(df_ICD_codes_vectorized_pca.shape)
df_ICD_codes_vectorized_pca.head(2)

(458328, 601)


,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30,PCA31,PCA32,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,PCA41,PCA42,PCA43,PCA44,PCA45,PCA46,PCA47,PCA48,PCA49,PCA50,PCA51,PCA52,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,PCA61,PCA62,PCA63,PCA64,PCA65,PCA66,PCA67,PCA68,PCA69,PCA70,PCA71,PCA72,PCA73,PCA74,PCA75,PCA76,PCA77,PCA78,PCA79,PCA80,PCA81,PCA82,PCA83,PCA84,PCA85,PCA86,PCA87,PCA88,PCA89,PCA90,PCA91,PCA92,PCA93,PCA94,PCA95,PCA96,PCA97,PCA98,PCA99,PCA100,PCA101,PCA102,PCA103,PCA104,PCA105,PCA106,PCA107,PCA108,PCA109,PCA110,PCA111,PCA112,PCA113,PCA114,PCA115,PCA116,PCA117,PCA118,PCA119,PCA120,PCA121,PCA122,PCA123,PCA124,PCA125,PCA126,PCA127,PCA128,PCA129,PCA130,PCA131,PCA132,PCA133,PCA134,PCA135,PCA136,PCA137,PCA138,PCA139,PCA140,PCA141,PCA142,PCA143,PCA144,PCA145,PCA146,PCA147,PCA148,PCA149,PCA150,PCA151,PCA152,PCA153,PCA154,PCA155,PCA156,PCA157,PCA158,PCA159,PCA160,PCA161,PCA162,PCA163,PCA164,PCA165,PCA166,PCA167,PCA168,PCA169,PCA170,PCA171,PCA172,PCA173,PCA174,PCA175,PCA176,PCA177,PCA178,PCA179,PCA180,PCA181,PCA182,PCA183,PCA184,PCA185,PCA186,PCA187,PCA188,PCA189,PCA190,PCA191,PCA192,PCA193,PCA194,PCA195,PCA196,PCA197,PCA198,PCA199,PCA200,PCA201,PCA202,PCA203,PCA204,PCA205,PCA206,PCA207,PCA208,PCA209,PCA210,PCA211,PCA212,PCA213,PCA214,PCA215,PCA216,PCA217,PCA218,PCA219,PCA220,PCA221,PCA222,PCA223,PCA224,PCA225,PCA226,PCA227,PCA228,PCA229,PCA230,PCA231,PCA232,PCA233,PCA234,PCA235,PCA236,PCA237,PCA238,PCA239,PCA240,PCA241,PCA242,PCA243,PCA244,PCA245,PCA246,PCA247,PCA248,PCA249,PCA250,...,PCA352,PCA353,PCA354,PCA355,PCA356,PCA357,PCA358,PCA359,PCA360,PCA361,PCA362,PCA363,PCA364,PCA365,PCA366,PCA367,PCA368,PCA369,PCA370,PCA371,PCA372,PCA373,PCA374,PCA375,PCA376,PCA377,PCA378,PCA379,PCA380,PCA381,PCA382,PCA383,PCA384,PCA385,PCA386,PCA387,PCA388,PCA389,PCA390,PCA391,PCA392,PCA393,PCA394,PCA395,PCA396,PCA397,PCA398,PCA399,PCA400,PCA401,PCA402,PCA403,PCA404,PCA405,PCA406,PCA407,PCA408,PCA409,PCA410,PCA411,PCA412,PCA413,PCA414,PCA415,PCA416,PCA417,PCA418,PCA419,PCA420,PCA421,PCA422,PCA423,PCA424,PCA425,PCA426,PCA427,PCA428,PCA429,PCA430,PCA431,PCA432,PCA433,PCA434,PCA435,PCA436,PCA437,PCA438,PCA439,PCA440,PCA441,PCA442,PCA443,PCA444,PCA445,PCA446,PCA447,PCA448,PCA449,PCA450,PCA451,PCA452,PCA453,PCA454,PCA455,PCA456,PCA457,PCA458,PCA459,PCA460,PCA461,PCA462,PCA463,PCA464,PCA465,PCA466,PCA467,PCA468,PCA469,PCA470,PCA471,PCA472,PCA473,PCA474,PCA475,PCA476,PCA477,PCA478,PCA479,PCA480,PCA481,PCA482,PCA483,PCA484,PCA485,PCA486,PCA487,PCA488,PCA489,PCA490,PCA491,PCA492,PCA493,PCA494,PCA495,PCA496,PCA497,PCA498,PCA499,PCA500,PCA501,PCA502,PCA503,PCA504,PCA505,PCA506,PCA507,PCA508,PCA509,PCA510,PCA511,PCA512,PCA513,PCA514,PCA515,PCA516,PCA517,PCA518,PCA519,PCA520,PCA521,PCA522,PCA523,PCA524,PCA525,PCA526,PCA527,PCA528,PCA529,PCA530,PCA531,PCA532,PCA533,PCA534,PCA535,PCA536,PCA537,PCA538,PCA539,PCA540,PCA541,PCA542,PCA543,PCA544,PCA545,PCA546,PCA547,PCA548,PCA549,PCA550,PCA551,PCA552,PCA553,PCA554,PCA555,PCA556,PCA557,PCA558,PCA559,PCA560,PCA561,PCA562,PCA563,PCA564,PCA565,PCA566,PCA567,PCA568,PCA569,PCA570,PCA571,PCA572,PCA573,PCA574,PCA575,PCA576,PCA577,PCA578,PCA579,PCA580,PCA581,PCA582,PCA583,PCA584,PCA585,PCA586,PCA587,PCA588,PCA589,PCA590,PCA591,PCA592,PCA593,PCA594,PCA595,PCA596,PCA597,PCA598,PCA599,PCA600,DESYNPUF_ID
0,3.647593,-0.782340,-0.935462,-0.046088,1.017342,1.831235,0.164655,-0.692281,0.362287,-0.577394,-0.369858,1.329667,-1.084824,0.006693,1.390489,-0.882084,-0.491234,-0.335683,0.696851,0.094493,-0.462910,0.064547,0.175329,-0.170556,1.008885,-1.333923,0.372988,0.194541,-0.762862,-0.084298,-0.061375,0.634967,-0.057567,-0.118257,0.080684,0.471468,0.235070,-0.206037,0.257731,-0.142542,-0.105404,-0.237514,-0.312377,-0.281149,-0.387043,-0.242147,-0.000794,-0.458357,0.435621,0.121936,-0.405941,-0.110268,-0.018663,-0.075231,-0.089828,-0.047926,0.274208,0.933219,-0.700989,0.909341,-0.372295,-0.288081,-

In [46]:
datetime.now()

datetime.datetime(2020, 5, 12, 16, 6, 29, 989872)

### Wrting this to a table in PostGres Database

In [47]:
# connect using sqlalchemy
import sqlalchemy

engine = sqlalchemy.engine.create_engine(conn_str)

In [48]:
pd.read_sql("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public'", con=engine)

,tablename
0,icd9_diagonsis
1,icd9_procedures
2,hcpcs
3,beneficiary2008
4,beneficiary2009
5,beneficiary2010
6,inpatient_claims
7,outpatient_claims
8,prescription_drug_events
9,final2008_2009


In [49]:
datetime.now()

datetime.datetime(2020, 5, 12, 16, 7, 38, 497249)

In [50]:
# Takes around xx mins for 4 folders data (450,000 rows & 600 columns)

#df_ICD_codes_vectorized.to_sql(name = 'ICD_Vectorized', con = engine, if_exists = 'replace', index = False)
df_ICD_codes_vectorized_pca.to_sql(name = 'ICD_Vectorized', con = engine, if_exists = 'replace', index = False)

KeyboardInterrupt: 

In [ ]:
datetime.now()

In [ ]:
df.to_sql(name = 'final2008_2009v2', con = engine, if_exists = 'replace', index = False)

In [ ]:
conn.commit()   #engine.commit()

In [51]:
pd.read_sql("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public'", con=engine)

,tablename
0,icd9_diagonsis
1,icd9_procedures
2,hcpcs
3,beneficiary2008
4,beneficiary2009
5,beneficiary2010
6,inpatient_claims
7,outpatient_claims
8,prescription_drug_events
9,final2008_2009


In [52]:
conn.close()